In [1]:
import numpy as np
import torch

from torch.optim import lr_scheduler
from skimage import io, transform
from matplotlib import pyplot as plt
import torchvision
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import os

## 1.1 PyTorch 张量    

PyTorch 的关键数据结构是张量，即多维数组。其功能与 NumPy 的 ndarray 对象类似，如下我们可以使用 torch.Tensor() 创建张量   

In [2]:
# Generate a 2-D pytorch tensor (i.e., a matrix)
pytorch_tensor = torch.Tensor(10, 20)
print("type: ", type(pytorch_tensor), "and size: ", pytorch_tensor.shape)

type:  <class 'torch.FloatTensor'> and size:  torch.Size([10, 20])


如果你需要一个兼容 NumPy 的表征，或者你想从现有的 NumPy 对象中创建一个 PyTorch 张量，那么就很简单了。   

In [3]:
# Convert the pytorch tensor to a numpy array:
numpy_tensor = pytorch_tensor.numpy()
print("type: ", type(numpy_tensor), "and size: ", numpy_tensor.shape)

# Convert the numpy array to Pytorch Tensor:
pytorch_tensor_from_numpy = torch.Tensor(numpy_tensor)
print("type: ", type(pytorch_tensor_from_numpy), "and size: ", pytorch_tensor_from_numpy.shape)

type:  <class 'numpy.ndarray'> and size:  (10, 20)
type:  <class 'torch.FloatTensor'> and size:  torch.Size([10, 20])


## 1.2 PyTorch vs. NumPy   

PyTorch 并不是 NumPy 的简单替代品，但它实现了很多 NumPy 功能。其中有一个不便之处是其命名规则，有时候它和 NumPy 的命名方法相当不同。我们来举几个例子说明其中的区别：     

In [4]:
# 1 张量创建 
t = torch.rand(2, 4, 3, 5)
a = np.random.rand(2,4,3,5)

# 2 张量分割  
a = t.numpy()
pytorch_slice = t[0, 1:3, :, 4]
numpy_slice = a[0, 1:3, :, 4]
print('Tensor[0, 1:3, :, 4]: \n', pytorch_slice)
print('NdArray[0, 1:3, :, 4]: \n', numpy_slice)

# 3 张量 Masking 
t = t- 0.5
a = t.numpy()
pytorch_masked = t[t > 0]
numpy_masked = a[a > 0]
print('\n\n')
print('Tensor Masked[ > 0]: \n', pytorch_masked)
print('NdArray Masked[ > 0]: \n', numpy_masked)

Tensor[0, 1:3, :, 4]: 
 
 0.1894  0.0223  0.6835
 0.1893  0.7526  0.5745
[torch.FloatTensor of size 2x3]

NdArray[0, 1:3, :, 4]: 
 [[0.18937165 0.02234507 0.6834594 ]
 [0.18930644 0.7526294  0.57448363]]



Tensor Masked[ > 0]: 
 
 0.0642
 0.1557
 0.2599
 0.1114
 0.3491
 0.4310
 0.4877
 0.4954
 0.4013
 0.4260
 0.0523
 0.2934
 0.2819
 0.4429
 0.0061
 0.1835
 0.3229
 0.3450
 0.0973
 0.4545
 0.2526
 0.3362
 0.1556
 0.1637
 0.0745
 0.4157
 0.0460
 0.4945
 0.1632
 0.2955
 0.0083
 0.3109
 0.3672
 0.4373
 0.1461
 0.1979
 0.3489
 0.2996
 0.3384
 0.3077
 0.1458
 0.3778
 0.2888
 0.2980
 0.4427
 0.4778
 0.4666
 0.0463
 0.1463
 0.3214
 0.2673
 0.2353
 0.3770
 0.4079
 0.0747
 0.4805
 0.3928
 0.1455
 0.1484
 0.1588
 0.1095
 0.1398
 0.1123
 0.3116
[torch.FloatTensor of size 64]

NdArray Masked[ > 0]: 
 [0.06424862 0.1557194  0.25991428 0.11136186 0.3491289  0.43095034
 0.4877271  0.49543792 0.40133327 0.42603815 0.05229646 0.29344392
 0.2818557  0.4429251  0.00611085 0.1834594  0.32294118 0.34498173


## 1.3 PyTorch 变量   

- PyTorch 张量的简单封装    
- 帮助建立计算图   
- Autograd（自动微分库）的必要部分   
- 将关于这些变量的梯度保存在 .grad 中    

![](https://mmbiz.qpic.cn/mmbiz_png/KmXPKA19gWic5fEM0JbibKGU6dqREn9g9ntLluCic3CHpXCJFicqORAbicHt4pD0sPG8FB8PxvJhShAaAP976raGaYA/640?wx_fmt=png&tp=webp&wxfrom=5&wx_lazy=1)

结构图:    

![](https://mmbiz.qpic.cn/mmbiz_png/KmXPKA19gWic5fEM0JbibKGU6dqREn9g9nwTAEvHBLXP48kXqAcS9icibm9HJPKdyUBricz9jvygicLic7FiblofSkA2Fg/640?wx_fmt=png&tp=webp&wxfrom=5&wx_lazy=1)

计算图和变量：在 PyTorch 中，神经网络会使用相互连接的变量作为计算图来表示。PyTorch 允许通过代码构建计算图来构建网络模型；之后 PyTorch 会简化估计模型权重的流程，例如通过自动计算梯度的方式。   

举例来说，假设我们想构建两层模型，那么首先要为输入和输出创建张量变量。我们可以将 PyTorch Tensor 包装进 Variable 对象中：   

In [5]:
from torch.autograd import Variable
import torch.nn.functional as F

x = Variable(torch.randn(4, 1), requires_grad = False)
y = Variable(torch.randn(2, 1), requires_grad = False)


我们把 requires_grad 设置为 True，表明我们想要自动计算梯度，这将用于反向传播中以优化权重。   

现在我们来定义权重：   

In [6]:
w1 = Variable(torch.randn(5, 4), requires_grad = True)
w2 = Variable(torch.randn(2, 5), requires_grad = True)

训练模型：   

In [7]:
def model_forward(x):
    return F.sigmoid(w2 @ F.sigmoid( w1 @ x))

# 打印初始化数据
print(w1)
print(w1.data.shape)
print(w1.grad)    # Initially, non-existent

Variable containing:
 0.0075 -0.1060  1.6483  1.3925
 0.9572  0.4237  0.2569 -0.0794
-0.9989  0.7801  1.3199 -0.1410
 1.0551 -0.7859  0.4443  1.6365
 1.7632  0.1500  0.6724 -0.3000
[torch.FloatTensor of size 5x4]

torch.Size([5, 4])
None


## 1.4 PyTorch 反向传播   

这样我们有了输入和目标、模型权重，那么是时候训练模型了。我们需要三个组件：   

(1) 损失函数：描述我们模型的预测距离目标还有多远；   

In [8]:
import torch.nn as nn
criterion = nn.MSELoss()

(2) 优化算法：用于更新权重；    

In [9]:
import torch.optim as optim
optimier = optim.SGD([w1, w2], lr = 0.001)

(3) 反向传播步骤：   

In [12]:
for epoch in range(100):
    loss = criterion(model_forward(x), y)
    optimier.zero_grad()   # Zero-out previous gradients
    loss.backward()    # Compute new gradients
    optimier.step()    # Apply these gradients
print (w1)

Variable containing:
 0.0077 -0.1059  1.6494  1.3938
 0.9574  0.4238  0.2580 -0.0779
-0.9990  0.7800  1.3193 -0.1417
 1.0551 -0.7859  0.4444  1.6366
 1.7632  0.1500  0.6723 -0.3001
[torch.FloatTensor of size 5x4]



## 1.5 PyTorch CUDA 接口   

PyTorch 的优势之一是为张量和 autograd 库提供 CUDA 接口。使用 CUDA GPU，你不仅可以加速神经网络训练和推断，还可以加速任何映射至 PyTorch 张量的工作负载。  

你可以调用 torch.cuda.is_available() 函数，检查 PyTorch 中是否有可用 CUDA。   

In [13]:
cuda_gpu = torch.cuda.is_available()
if cuda_gpu:
    print("Great, you have a GPU!")
else:
    print("Life is short -- consider a GPU!")

Great, you have a GPU!


**.cuda()**   
很好，现在你有 GPU 了。      

之后，使用 cuda 加速代码就和调用一样简单。如果你在张量上调用 .cuda()，则它将执行从 CPU 到 CUDA GPU 的数据迁移。如果你在模型上调用 .cuda()，则它不仅将所有内部储存移到 GPU，还将整个计算图映射至 GPU。    

要想将张量或模型复制回 CPU，比如想和 NumPy 交互，你可以调用 .cpu()。    